---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
import pandas as pd
import numpy as np

In [3]:
median_home_price=pd.read_csv('City_Zhvi_AllHomes.csv')

In [4]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
median_home_price['State']=median_home_price['State'].replace(states).astype(object)

In [5]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
###############################################################################    
    # Firstly, read the txt file.
    university_town=pd.read_csv('university_towns.txt', sep="\t")
    
    
    #
    line=pd.DataFrame({'Alabama[edit]':'Alabama[edit]'},index=[0])
    university_town=pd.concat([university_town.iloc[:0],line,university_town.iloc[0:]]).reset_index(drop=True)
    
    
    university_town=university_town.rename(columns={'Alabama[edit]':0}) 
    
    df2=university_town
    
    s=df2[(df2[0].str.endswith('[edit]'))]
    s[0]=s[0].str.slice(0,-6)
    k=list(s.index)
    
    
    df2[0]=df2[0].str.split('(')
    
    l=[]
    for i in range(len(df2)):
        l.append(df2.iloc[i][0][0])
        
    for i in range(len(l)):
        l[i]=l[i].strip()
    
    j=0
    km=[]
    for i in range(len(l)):
        if i==k[j]:
            state=l[k[j]]
            j=j+1
            if j==len(k):
                while i+1 != len(l):
                    i=i+1
                    km.append([state,l[i]])
                break
                
        else:
            km.append([state,l[i]])
        
    university_df=pd.DataFrame(data=km,columns=['State','RegionName'])
    university_df['State']=university_df['State'].str.slice(0,-6)
    
    return university_df

get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [6]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    gdplev=pd.read_excel('gdplev.xls',skiprows=7)
    gdplev=gdplev.reset_index()
    gdplev=gdplev.iloc[212:][['Unnamed: 4','Unnamed: 5']]
    gdplev.columns = ['Quarter','GDP']
    gdplev=gdplev.reset_index()
    gdplev=gdplev[['Quarter','GDP']]
    
    for i in range(2,len(gdplev)):
        if (gdplev.GDP[i-2]>gdplev.GDP[i-1] and gdplev.GDP[i-1]>gdplev.GDP[i]):
            a=(gdplev.Quarter[i-2])
            break
    
    
    return str(a)

get_recession_start()

'2008q3'

In [11]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
    gdplev=pd.read_excel('gdplev.xls',skiprows=7)
    gdplev=gdplev.reset_index()
    gdplev=gdplev.iloc[212:][['Unnamed: 4','Unnamed: 5']]
    gdplev.columns = ['Quarter','GDP']
    gdplev=gdplev.reset_index()
    gdplev=gdplev[['Quarter','GDP']]
    
    for i in range(4,len(gdplev)):
        if (gdplev.GDP[i-4] > gdplev.GDP[i-3] and gdplev.GDP[i-3] > gdplev.GDP[i-2] and gdplev.GDP[i-2] < gdplev.GDP[i-1]
       and gdplev.GDP[i-1] < gdplev.GDP[i]):
            a=(gdplev.Quarter[i])
    
     
    return str(a)
get_recession_end()

'2009q4'

In [8]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    gdplev=pd.read_excel('gdplev.xls',skiprows=7)
    gdplev=gdplev.reset_index()
    gdplev=gdplev.iloc[212:][['Unnamed: 4','Unnamed: 5']]
    gdplev.columns = ['Quarter','GDP']
    gdplev=gdplev.reset_index()
    gdplev=gdplev[['Quarter','GDP']]
    recession=[]
    for i in range(4,len(gdplev)):
        if (gdplev.GDP[i-4]>gdplev.GDP[i-3] and gdplev.GDP[i-3]>gdplev.GDP[i-2] and
            gdplev.GDP[i-2] <gdplev.GDP[i-1] and gdplev.GDP[i-1] <gdplev.GDP[i]):
            
            break
    return str(gdplev.Quarter[i-2])
get_recession_bottom()

'2009q2'

In [12]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    housing_data=pd.read_csv('City_Zhvi_AllHomes.csv')
    
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    housing_data['State']=housing_data['State'].replace(states).astype(object)
    
    k=list(housing_data.columns)
    k=k[51:]
    k.insert(0,'State')
    k.insert(1,'RegionName')
    
    
    
    col=k
    col=col[2:]
    df=housing_data
    
    mean=[]
    for i in range(len(col)):
        if (i+1)%3==0:
            a=np.array(housing_data[[col[i-2],col[i-1],col[i]]])
            a=a.T
            M=np.mean(a,0)
            M=M.T
            if int(col[i][-2:])/3==1.0:
                df[str(col[i][:4])+'q1']=M
            if int(col[i][-2:])/3==2:
                df[str(col[i][:4])+'q2']=M
            if int(col[i][-2:])/3==3:
                df[str(col[i][:4])+'q3']=M
            if int(col[i][-2:])/3==4:
                df[str(col[i][:4])+'q4']=M
        if i+1==len(col):
            a=np.array(housing_data[[col[i-1],col[i]]])
            a=a.T
            M=np.mean(a,0)
            M=M.T
            df[str(col[i][:4])+'q3']=M
       
    
    df=df.T
    df=df[-67:]
    df=df.T
    df[['State','RegionName']]=housing_data[['State','RegionName']]
    df=df.set_index(['State','RegionName'])
    
    
    
    
    return df


convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515467,522800,528067,532267,540800,557200,572833,582867,591633,587200
California,Los Angeles,207067,214467,220967,226167,233000,239100,245067,253033,261967,272700,...,498033,509067,518867,528800,538167,547267,557733,566033,577467,584050
Illinois,Chicago,138400,143633,147867,152133,156933,161800,166400,170433,175500,177567,...,192633,195767,201267,201067,206033,208300,207900,206067,208200,212000
Pennsylvania,Philadelphia,53000,53633.3,54133.3,54700,55333.3,55533.3,56266.7,57533.3,59133.3,60733.3,...,113733,115300,115667,116200,117967,121233,122200,123433,126933,128700
Arizona,Phoenix,111833,114367,116000,117400,119600,121567,122700,124300,126533,128367,...,164267,165367,168500,171533,174167,179067,183833,187900,191433,195200
Nevada,Las Vegas,132600,134367,135400,137000,139533,141733,143367,146133,149333,150933,...,170067,173400,175467,177500,181600,186767,190633,194600,197200,199950
California,San Diego,222900,234367,245433,256033,267200,276267,284500,291933,301233,312867,...,480200,489033,496433,503367,512067,519767,525467,529333,536233,539750
Texas,Dallas,84466.7,83866.7,84866.7,87833.3,89733.3,89300,89066.7,90900,92566.7,93800,...,106633,108900,111533,113700,121133,128567,134600,140500,144600,149300
California,San Jose,374267,406567,431867,455500,470667,470200,456800,445567,441433,457767,...,679400,697033,714933,731433,756733,776400,789133,803600,818933,822200


In [13]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    from scipy.stats import ttest_ind
    
    
    df=convert_housing_data_to_quarters()
    df=df.reset_index()
    
    housing_data=pd.read_csv('City_Zhvi_AllHomes.csv')
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    housing_data['State']=housing_data['State'].replace(states).astype(object)
    
    df[['State','RegionName']]=housing_data[['State','RegionName']]
    
    
    university_df=get_list_of_university_towns()
    university_df['uni']=True
    
    uni_town=pd.merge(university_df,df,how='inner',on=['State','RegionName'])
    all_town=pd.merge(uni_town,df,how='outer',on=['State','RegionName'])
    
    all_town=all_town.rename(columns={'2008q3_y':'2008q3','2008q4_y':'2008q4',
                                 '2009q1_y':'2009q1','2009q2_y':'2009q2'})
    
    k=list(all_town.columns)
    a=k[0:3]
    b=k[-4:]
    
    all_town=all_town[k]
    
    
    recession=['State','RegionName','2008q3','2008q4', '2009q1', '2009q2']
    
    df=df[recession]
    

    
    all_town['uni']=all_town['uni'].replace({np.NaN:False})
    
    all_town['PriceRatio']=all_town['2009q2']-all_town['2008q3']
    
    uni_town_price=all_town[(all_town['uni']==True)]
    non_uni_town_price=all_town[(all_town['uni']==False)]
    
    
    uni_town_price_ratio=uni_town_price['PriceRatio']
    non_uni_town_price_ratio=non_uni_town_price['PriceRatio']

    
    
    
    st, p = ttest_ind(uni_town_price_ratio.dropna(), non_uni_town_price_ratio.dropna())

    
    if p < 0.01:
        different=True
    else:
        different=False
        
    
    better=''
    if uni_town_price_ratio.mean() > non_uni_town_price_ratio.mean():
        better='university town'
    else:
        better='non-university town'
        
    return (different,p,better)

run_ttest()

(True, 0.0045824516144303247, 'university town')